In [7]:
import io
import json
import os
import sys
import random
import csv
import subprocess
import numpy as np
import pandas as pd
import tifffile as tiff
from PIL import Image
from math import sqrt, exp, log
from matplotlib import cm
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import shutil
from flir_image_extractor import FlirImageExtractor

In [10]:
os.chdir(sys.path[0])
rgbt_jpg_source_dir = os.path.join(
    'raw_data_source', 'RGBT_jpg')
t_npy_dest_dir = os.path.join(
    'raw_data_source', 'T_jpg')
rgb_npy_dest_dir = os.path.join(
    'raw_data_source', 'RGB_npy')

rgbt_jpg_file_list = sorted([os.path.join(
    rgbt_jpg_source_dir, n) for n in os.listdir(rgbt_jpg_source_dir) if not n.startswith('.') and n.endswith('.jpg')])

In [ ]:
fie = FlirImageExtractor(exiftool_path='exiftool')

raw_dataset = [] 

for i, rgbt_file in tqdm(enumerate(rgbt_jpg_file_list)):
    try:
        fie.process_image(rgbt_file)
        thermal_channel = fie.get_thermal_np()
        rgb_channels = fie.get_rgb_np()
        meta = fie.get_metadata()
        json_string = json.dumps(meta, indent=4).replace('null', 'None')
        if i%20 == 0:
            print(f'thermal_channel.dtype == {thermal_channel.dtype}')
            print(f'thermal_channel.shape == {thermal_channel.shape}')
            print(f'rgb_channels.dtype == {rgb_channels.dtype}')
            print(f'rgb_channels.shape == {rgb_channels.shape}')
            print(f'{json_string}')
        dest_npy_file = os.path.join(t_npy_dest_dir, f'{Path(rgbt_file).stem}.npy')
        np.save(dest_npy_file, thermal_channel)
        print(f'{dest_npy_file} saved succesfully.')
        meta["t_numpy_file"] = dest_npy_file
        dest_npy_file = os.path.join(rgb_npy_dest_dir, f'{Path(rgbt_file).stem}.npy')
        np.save(dest_npy_file, rgb_channels)
        print(f'{dest_npy_file} saved succesfully.')
        meta["rgb_numpy_file"] = dest_npy_file
        raw_dataset.append(meta)

    except Exception as e:
        print(f"An error occurred: {e}")

df = pd.DataFrame(raw_dataset)

# Save the DataFrame to a CSV file
df.to_csv('raw_data.csv', index=False)
    